In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os
api_key = os.getenv("OPENAI_API_KEY")

In [3]:
from openai import OpenAI

client = OpenAI(api_key=api_key)

In [4]:
import json
from tqdm import tqdm

In [5]:
input_data = "./data/extracted_requirements_final.json"

In [6]:
def generate_detailed_prompt(description, snippet, module):
    return f"""
다음은 소프트웨어 시스템의 간략한 요구사항 정보입니다.

[요구사항 설명]: {description}
[원문 요약]: {snippet}
[담당 모듈]: {module}

아래 형식에 따라 이 요구사항에 대한 상세 설명을 작성해 주세요.

□ 요구사항은 아래 내용을 참조로 도출하며, 요구사항, 대상업무, 요건처리 상세의 형식으로 작성 사례를 참조하여 작성함
1) 요구사항 도출을 위한 기초자료
   - PI 사업을 통해 도출된 산출물
   - 현행 시스템을 분석한 현행시스템분석서
   - 사업수행계획서, RFP, 인터뷰 결과서

2) 작성 방법 (신규/개선 요구사항 기준)
[요구사항]
- 요구사항 내용을 간결하게 기술합니다.
[대상업무]
- 요구사항이 반영될 대상 업무를 기술하고, 구현되는 기능 유형(예: 화면, 온라인, 배치 등)을 명시합니다.
[요건처리 상세]
- 해당 요구사항을 어떻게 처리할지 구체적으로 기술합니다. (예: 어떤 데이터를 어떤 기준으로 계산하고 어디에 표시하는지, 흐름 중심의 처리 방식 등)

예시:
[요구사항]
영업부점에서 등록한 지준이체예상보고 데이터를 기준으로 입출금 내역을 집계한다.
[대상업무]
자금결제관리 - 화면 기능
[요건처리 상세]
지준이체예상보고서에 등록된 입금/출금 데이터를 수집하여 이체사유별로 당일 집계금액을 계산하고, 사용자가 조회 가능한 화면에 표출한다.

※ 반드시 위와 같은 형식으로만 응답해 주세요.
"""

In [7]:
def get_detailed_description(description, snippet, module):
    prompt = generate_detailed_prompt(description, snippet, module)
    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "당신은 시스템 분석 전문가이며, 업무 설명을 상세하게 기술하는 역할입니다."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.3
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print("Error:", e)
        return "상세 설명 생성 실패"

In [8]:
# JSON 불러오기
with open(input_data, "r", encoding="utf-8") as f:
    requirements = json.load(f)

In [ ]:
# 상세 설명 생성
for req in tqdm(requirements, desc="🔍 상세 설명 생성 중"):
    desc = req.get("description", "")
    snippet = req.get("raw_text_snippet", "")
    module = req.get("responsible_module", "")
    req["detailed_description"] = get_detailed_description(desc, snippet, module)

# 결과 저장
detailed_output_path = "./data/enriched_requirements_detailed.json"
with open(detailed_output_path, "w", encoding="utf-8") as f:
    json.dump(requirements, f, ensure_ascii=False, indent=4)

print(f"✅ 상세 설명이 생성되어 저장되었습니다: {detailed_output_path}")